**Описание процекта: Мне предстоит оптимизировать производственные расходы, металлургический комбинат ООО «Так закаляем сталь».**

### Описание этапа обработки

Сталь обрабатывают в металлическом ковше вместимостью около 100 тонн. Чтобы ковш выдерживал высокие температуры, изнутри его облицовывают огнеупорным кирпичом. Расплавленную сталь заливают в ковш и подогревают до нужной температуры графитовыми электродами. Они установлены в крышке ковша. 

Из сплава выводится сера (десульфурация), добавлением примесей корректируется химический состав и отбираются пробы. Сталь легируют — изменяют её состав — подавая куски сплава из бункера для сыпучих материалов или проволоку через специальный трайб-аппарат (англ. tribe, «масса»).

Перед тем как первый раз ввести легирующие добавки, измеряют температуру стали и производят её химический анализ. Потом температуру на несколько минут повышают, добавляют легирующие материалы и продувают сплав инертным газом. Затем его перемешивают и снова проводят измерения. Такой цикл повторяется до достижения целевого химического состава и оптимальной температуры плавки.

Тогда расплавленная сталь отправляется на доводку металла или поступает в машину непрерывной разливки. Оттуда готовый продукт выходит в виде заготовок-слябов (англ. *slab*, «плита»).

### Описание данных

Данные состоят из файлов, полученных из разных источников:

- `data_arc.csv` — данные об электродах;
- `data_bulk.csv` — данные о подаче сыпучих материалов (объём);
- `data_bulk_time.csv` *—* данные о подаче сыпучих материалов (время);
- `data_gas.csv` — данные о продувке сплава газом;
- `data_temp.csv` — результаты измерения температуры;
- `data_wire.csv` — данные о проволочных материалах (объём);
- `data_wire_time.csv` — данные о проволочных материалах (время).

Во всех файлах столбец `key` содержит номер партии. В файлах может быть несколько строк с одинаковым значением `key`: они соответствуют разным итерациям обработки.

**План работы**

**Наша главная задача - предсказать финальную температуру. Исходя из данной цели мы выполним следующие задачи:**
- Проанализировать наши данные
- Обработать данные
- Сгенерировать дополнительные параметры
- Создать единый датафрейм для дальнейшей работы
- Разделить датафрейм на фичи/таргет и обучающую/валидационную выборки
- Протестировать на обчучающей выборке различные модели и определить лучшую.

Для начала мы загружаем библиотеки для дальнейшего использования

<font color='steelblue'><b>Комментарий тимлида</b></font><br>
<font color='green'>✔️ Описание на месте 👍🏼 можно ещё цели добавить.</font><br>

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
from statsmodels.tsa.seasonal import seasonal_decompose
import time
import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import statistics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostRegressor
from sklearn import svm, datasets
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from lightgbm import LGBMClassifier, LGBMRegressor
import lightgbm as lgb

Ниже мы загружаем данные для дальнейшего анализа

**Данные из тренажера**

In [2]:
#данные об электродах;
data_arc = pd.read_csv('/datasets/final_steel/data_arc.csv')
#данные о подаче сыпучих материалов (объём);
data_bulk = pd.read_csv('/datasets/final_steel/data_bulk.csv')

#данные о подаче сыпучих материалов (время);
data_bulk_time = pd.read_csv('/datasets/final_steel/data_bulk_time.csv')

#данные о продувке сплава газом;
data_gas = pd.read_csv('/datasets/final_steel/data_gas.csv')

#результаты измерения температуры;
data_temp = pd.read_csv('/datasets/final_steel/data_temp.csv')

#данные о проволочных материалах (объём);
data_wire = pd.read_csv('/datasets/final_steel/data_wire.csv')

#данные о проволочных материалах (время)
data_wire_time = pd.read_csv('/datasets/final_steel/data_wire_time.csv')

**Проводим первичный визуальный анализ**

**1) Данные об электродах**

In [3]:
print('Данные об электродах')
display(data_arc)
print('')
data_arc.info()
print('')
print(data_arc.isna().sum())
print('')
print(data_arc.describe())
print('_____________________________________________________')


Данные об электродах


,key,Начало нагрева дугой,Конец нагрева дугой,Активная мощность,Реактивная мощность
0,1,2019-05-03 11:02:14,2019-05-03 11:06:02,0.976059,0.687084
1,1,2019-05-03 11:07:28,2019-05-03 11:10:33,0.805607,0.520285
2,1,2019-05-03 11:11:44,2019-05-03 11:14:36,0.744363,0.498805
3,1,2019-05-03 11:18:14,2019-05-03 11:24:19,1.659363,1.062669
4,1,2019-05-03 11:26:09,2019-05-03 11:28:37,0.692755,0.414397
...,...,...,...,...,...
14871,3241,2019-09-01 03:58:58,2019-09-01 04:01:35,0.533670,0.354439
14872,3241,2019-09-01 04:05:04,2019-09-01 04:08:04,0.676604,0.523631
14873,3241,2019-09-01 04:16:41,2019-09-01 04:19:45,0.733899,0.475654
14874,3241,2019-09-01 04:31:51,2019-09-01 04:32:48,0.220694,0.145768



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14876 entries, 0 to 14875
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   key                   14876 non-null  int64  
 1   Начало нагрева дугой  14876 non-null  object 
 2   Конец нагрева дугой   14876 non-null  object 
 3   Активная мощность     14876 non-null  float64
 4   Реактивная мощность   14876 non-null  float64
dtypes: float64(2), int64(1), object(2)
memory usage: 581.2+ KB

key                     0
Начало нагрева дугой    0
Конец нагрева дугой     0
Активная мощность       0
Реактивная мощность     0
dtype: int64

                key  Активная мощность  Реактивная мощность
count  14876.000000       14876.000000         14876.000000
mean    1615.220422           0.670441             0.452592
std      934.571502           0.408159             5.878702
min        1.000000           0.030002          -715.504924
25%      806.000000      

1) Мы видим в реактивной мощности отрицательное значение. Необходимо привести все  значения выше нуля.

In [4]:
data_arc = data_arc[data_arc['Реактивная мощность'] > 0 ]

**2) Данные о подаче сыпучих материалов (объём)**

In [5]:

print('')
print('Данные о подаче сыпучих материалов (объём)')
display(data_bulk)
data_bulk.info()
print('')
print('')
print(data_bulk.isna().sum())
print(data_bulk.describe())
print('_____________________________________________________')



Данные о подаче сыпучих материалов (объём)


,key,Bulk 1,Bulk 2,Bulk 3,Bulk 4,Bulk 5,Bulk 6,Bulk 7,Bulk 8,Bulk 9,Bulk 10,Bulk 11,Bulk 12,Bulk 13,Bulk 14,Bulk 15
0,1,NaN,NaN,NaN,43.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,206.0,NaN,150.0,154.0
1,2,NaN,NaN,NaN,73.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,206.0,NaN,149.0,154.0
2,3,NaN,NaN,NaN,34.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,205.0,NaN,152.0,153.0
3,4,NaN,NaN,NaN,81.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,207.0,NaN,153.0,154.0
4,5,NaN,NaN,NaN,78.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,203.0,NaN,151.0,152.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3124,3237,NaN,NaN,170.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,252.0,NaN,130.0,206.0
3125,3238,NaN,NaN,126.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,254.0,NaN,108.0,106.0
3126,3239,NaN,NaN,NaN,NaN,NaN,114.0,NaN,NaN,NaN,NaN,NaN,158.0,NaN,270.0,88.0
3127,3240,NaN,NaN,NaN,NaN,NaN,26.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,192.0,54.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3129 entries, 0 to 3128
Data columns (total 16 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   key      3129 non-null   int64  
 1   Bulk 1   252 non-null    float64
 2   Bulk 2   22 non-null     float64
 3   Bulk 3   1298 non-null   float64
 4   Bulk 4   1014 non-null   float64
 5   Bulk 5   77 non-null     float64
 6   Bulk 6   576 non-null    float64
 7   Bulk 7   25 non-null     float64
 8   Bulk 8   1 non-null      float64
 9   Bulk 9   19 non-null     float64
 10  Bulk 10  176 non-null    float64
 11  Bulk 11  177 non-null    float64
 12  Bulk 12  2450 non-null   float64
 13  Bulk 13  18 non-null     float64
 14  Bulk 14  2806 non-null   float64
 15  Bulk 15  2248 non-null   float64
dtypes: float64(15), int64(1)
memory usage: 391.2 KB


key           0
Bulk 1     2877
Bulk 2     3107
Bulk 3     1831
Bulk 4     2115
Bulk 5     3052
Bulk 6     2553
Bulk 7     3104
Bulk 8     3128
Bulk 9    

Мы видим аномальные значение в колонке "Bulk 8". Данные неоднозначыне, только одно использование. Принимаем решение о удалении данной колонки.

In [6]:
data_bulk = data_bulk.drop('Bulk 8', axis=1)

**3) Данные о подаче сыпучих материалов (время)**

In [7]:
print('')
print('Данные о подаче сыпучих материалов (время)')
display(data_bulk_time)
data_bulk_time.info()
print(data_bulk_time.isna().sum())
print(data_bulk_time.describe())
print('_____________________________________________________')


Данные о подаче сыпучих материалов (время)


,key,Bulk 1,Bulk 2,Bulk 3,Bulk 4,Bulk 5,Bulk 6,Bulk 7,Bulk 8,Bulk 9,Bulk 10,Bulk 11,Bulk 12,Bulk 13,Bulk 14,Bulk 15
0,1,NaN,NaN,NaN,2019-05-03 11:21:30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-05-03 11:03:52,NaN,2019-05-03 11:03:52,2019-05-03 11:03:52
1,2,NaN,NaN,NaN,2019-05-03 11:46:38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-05-03 11:40:20,NaN,2019-05-03 11:40:20,2019-05-03 11:40:20
2,3,NaN,NaN,NaN,2019-05-03 12:31:06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-05-03 12:09:40,NaN,2019-05-03 12:09:40,2019-05-03 12:09:40
3,4,NaN,NaN,NaN,2019-05-03 12:48:43,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-05-03 12:41:24,NaN,2019-05-03 12:41:24,2019-05-03 12:41:24
4,5,NaN,NaN,NaN,2019-05-03 13:18:50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-05-03 13:12:56,NaN,2019-05-03 13:12:56,2019-05-03 13:12:56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3124,3237,NaN,NaN,2019-08-31 22:51:28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-08-31 22:46:52,NaN,2019-08-31 22:46:52,2019-08-31 22:46:52
3125,3238,NaN,NaN,2019-08-31 23:39:11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-08-31 23:33:09,NaN,2019-08-31 23:33:09,2019-08-31 23:33:09
3126,3239,NaN,NaN,NaN,NaN,NaN,2019-09-01 01:51:58,NaN,NaN,NaN,NaN,NaN,2019-09-01 01:39:41,NaN,2019-09-01 01:33:25,2019-09-01 01:33:25
3127,3240,NaN,NaN,NaN,NaN,NaN,2019-09-01 03:12:40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-09-01 02:41:27,2019-09-01 02:41:27


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3129 entries, 0 to 3128
Data columns (total 16 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   key      3129 non-null   int64 
 1   Bulk 1   252 non-null    object
 2   Bulk 2   22 non-null     object
 3   Bulk 3   1298 non-null   object
 4   Bulk 4   1014 non-null   object
 5   Bulk 5   77 non-null     object
 6   Bulk 6   576 non-null    object
 7   Bulk 7   25 non-null     object
 8   Bulk 8   1 non-null      object
 9   Bulk 9   19 non-null     object
 10  Bulk 10  176 non-null    object
 11  Bulk 11  177 non-null    object
 12  Bulk 12  2450 non-null   object
 13  Bulk 13  18 non-null     object
 14  Bulk 14  2806 non-null   object
 15  Bulk 15  2248 non-null   object
dtypes: int64(1), object(15)
memory usage: 391.2+ KB
key           0
Bulk 1     2877
Bulk 2     3107
Bulk 3     1831
Bulk 4     2115
Bulk 5     3052
Bulk 6     2553
Bulk 7     3104
Bulk 8     3128
Bulk 9     3110
Bulk 10    295

Отклонений не выявлено

**4) Данные о продувке сплава газом**

In [8]:
print('')
print('Данные о продувке сплава газом')
display(data_gas)
data_gas.info()
print(data_gas.isna().sum())
print(data_gas.describe())
print('_____________________________________________________')


Данные о продувке сплава газом


,key,Газ 1
0,1,29.749986
1,2,12.555561
2,3,28.554793
3,4,18.841219
4,5,5.413692
...,...,...
3234,3237,5.543905
3235,3238,6.745669
3236,3239,16.023518
3237,3240,11.863103


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3239 entries, 0 to 3238
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   key     3239 non-null   int64  
 1   Газ 1   3239 non-null   float64
dtypes: float64(1), int64(1)
memory usage: 50.7 KB
key      0
Газ 1    0
dtype: int64
               key        Газ 1
count  3239.000000  3239.000000
mean   1621.861377    11.002062
std     935.386334     6.220327
min       1.000000     0.008399
25%     812.500000     7.043089
50%    1622.000000     9.836267
75%    2431.500000    13.769915
max    3241.000000    77.995040
_____________________________________________________


Отклонений не выявлено

**5) Результаты измерения температуры**

In [9]:
print('')
print('Результаты измерения температуры')
display(data_temp)
data_temp.info()
print(data_temp.isna().sum())
print(data_temp.describe())
print('_____________________________________________________')


Результаты измерения температуры


,key,Время замера,Температура
0,1,2019-05-03 11:16:18,1571.0
1,1,2019-05-03 11:25:53,1604.0
2,1,2019-05-03 11:29:11,1618.0
3,1,2019-05-03 11:30:01,1601.0
4,1,2019-05-03 11:30:39,1613.0
...,...,...,...
15902,3241,2019-09-01 04:03:30,1586.0
15903,3241,2019-09-01 04:16:12,NaN
15904,3241,2019-09-01 04:22:39,NaN
15905,3241,2019-09-01 04:33:42,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15907 entries, 0 to 15906
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   key           15907 non-null  int64  
 1   Время замера  15907 non-null  object 
 2   Температура   13006 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 372.9+ KB
key                0
Время замера       0
Температура     2901
dtype: int64
                key   Температура
count  15907.000000  13006.000000
mean    1607.880870   1591.840920
std      942.212073     21.375851
min        1.000000   1191.000000
25%      790.000000   1581.000000
50%     1618.000000   1591.000000
75%     2427.000000   1601.000000
max     3241.000000   1705.000000
_____________________________________________________


Отклонений не выявлено

**6) Данные о проволочных материалах (объём)**

In [10]:
print('')
print('Данные о проволочных материалах (объём)')
display(data_wire)
data_wire.info()
print(data_wire.isna().sum())
print(data_wire.describe())
print('_____________________________________________________')


Данные о проволочных материалах (объём)


,key,Wire 1,Wire 2,Wire 3,Wire 4,Wire 5,Wire 6,Wire 7,Wire 8,Wire 9
0,1,60.059998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,96.052315,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,91.160157,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,89.063515,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,89.238236,9.11456,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
3076,3237,38.088959,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3077,3238,56.128799,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3078,3239,143.357761,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3079,3240,34.070400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3081 entries, 0 to 3080
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   key     3081 non-null   int64  
 1   Wire 1  3055 non-null   float64
 2   Wire 2  1079 non-null   float64
 3   Wire 3  63 non-null     float64
 4   Wire 4  14 non-null     float64
 5   Wire 5  1 non-null      float64
 6   Wire 6  73 non-null     float64
 7   Wire 7  11 non-null     float64
 8   Wire 8  19 non-null     float64
 9   Wire 9  29 non-null     float64
dtypes: float64(9), int64(1)
memory usage: 240.8 KB
key          0
Wire 1      26
Wire 2    2002
Wire 3    3018
Wire 4    3067
Wire 5    3080
Wire 6    3008
Wire 7    3070
Wire 8    3062
Wire 9    3052
dtype: int64
               key       Wire 1       Wire 2      Wire 3      Wire 4  Wire 5  \
count  3081.000000  3055.000000  1079.000000   63.000000   14.000000   1.000   
mean   1623.426485   100.895853    50.577323  189.482681   57.442841  15.1

Мы видим в колонке 'Wire 5' единичное значение. Предполагаем искажение данных/ошибку - удаляем колонку.

In [11]:
data_wire = data_wire.drop('Wire 5', axis=1)

**7) Данные  о проволочных материалах (время)**

In [12]:
print('')
print('Данные  о проволочных материалах (время)')
display(data_wire_time)
data_wire_time.info()
print(data_wire_time.isna().sum())
print(data_wire_time.describe())


Данные  о проволочных материалах (время)


,key,Wire 1,Wire 2,Wire 3,Wire 4,Wire 5,Wire 6,Wire 7,Wire 8,Wire 9
0,1,2019-05-03 11:11:41,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,2019-05-03 11:46:10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,2019-05-03 12:13:47,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,2019-05-03 12:48:05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,2019-05-03 13:18:15,2019-05-03 13:32:06,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
3076,3237,2019-08-31 22:50:20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3077,3238,2019-08-31 23:38:24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3078,3239,2019-09-01 01:50:43,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3079,3240,2019-09-01 03:12:02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3081 entries, 0 to 3080
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   key     3081 non-null   int64 
 1   Wire 1  3055 non-null   object
 2   Wire 2  1079 non-null   object
 3   Wire 3  63 non-null     object
 4   Wire 4  14 non-null     object
 5   Wire 5  1 non-null      object
 6   Wire 6  73 non-null     object
 7   Wire 7  11 non-null     object
 8   Wire 8  19 non-null     object
 9   Wire 9  29 non-null     object
dtypes: int64(1), object(9)
memory usage: 240.8+ KB
key          0
Wire 1      26
Wire 2    2002
Wire 3    3018
Wire 4    3067
Wire 5    3080
Wire 6    3008
Wire 7    3070
Wire 8    3062
Wire 9    3052
dtype: int64
               key
count  3081.000000
mean   1623.426485
std     932.996726
min       1.000000
25%     823.000000
50%    1619.000000
75%    2434.000000
max    3241.000000


Отклонений не выявлено

**Длительность времени между первым и последним замером температуры + вычислите статистики + первая температура.**

Создаём рабочу таблицу для манипуляций

In [13]:
display(data_temp)

,key,Время замера,Температура
0,1,2019-05-03 11:16:18,1571.0
1,1,2019-05-03 11:25:53,1604.0
2,1,2019-05-03 11:29:11,1618.0
3,1,2019-05-03 11:30:01,1601.0
4,1,2019-05-03 11:30:39,1613.0
...,...,...,...
15902,3241,2019-09-01 04:03:30,1586.0
15903,3241,2019-09-01 04:16:12,NaN
15904,3241,2019-09-01 04:22:39,NaN
15905,3241,2019-09-01 04:33:42,NaN


Ниже мы извлекаем уникальные key из каждой таблицы и сводим в одну таблицу, где встречаются только пересекающиеся значения.

In [14]:
work_data = pd.DataFrame([])
work_data['key'] = data_temp['key'].unique()
data_arc_key = pd.DataFrame([])
data_arc_key['key'] = data_arc['key'].unique()
work_data = work_data.merge(data_arc_key, on='key', how='inner')
data_bulk_key = pd.DataFrame([])
data_bulk_key['key'] = data_bulk['key'].unique()
work_data = work_data.merge(data_bulk_key, on='key', how='inner')
data_bulk_time_key = pd.DataFrame([])
data_bulk_time_key['key'] = data_bulk_time['key'].unique()
work_data = work_data.merge(data_bulk_time_key, on='key', how='inner')
data_gas_key = pd.DataFrame([])
data_gas_key['key'] = data_gas['key'].unique()
work_data = work_data.merge(data_gas_key, on='key', how='inner')
data_temp_key = pd.DataFrame([])
data_temp_key['key'] = data_temp['key'].unique()
work_data = work_data.merge(data_temp_key, on='key', how='inner')
data_wire_key = pd.DataFrame([])
data_wire_key['key'] = data_wire['key'].unique()
work_data = work_data.merge(data_wire_key, on='key', how='inner')
data_wire_time_key = pd.DataFrame([])
data_wire_time_key['key'] = data_wire_time['key'].unique()
work_data = work_data.merge(data_wire_time_key, on='key', how='inner')
work_data = work_data.rename(columns={"key": "key_xxx"})

In [16]:
work_data = work_data.merge(data_temp.groupby('key').min()['Время замера'], left_on='key_xxx', right_on='key')     
work_data = work_data.rename(columns={"Время замера": "Время замера MIN"})
work_data = work_data.merge(data_temp, left_on='Время замера MIN', right_on='Время замера') 
work_data = work_data.rename(columns={"Температура": "Температура 1"})
work_data = work_data.drop('Время замера', axis = 1)
work_data = work_data.merge(data_temp.groupby('key').max()['Время замера'], left_on='key_xxx', right_on='key')     
work_data = work_data.rename(columns={"Время замера": "Время замера MAX"})

In [17]:
work_data = work_data.merge(data_temp, left_on='Время замера MAX', right_on='Время замера') 
work_data = work_data.rename(columns={"Температура": "Температура 2"})
work_data = work_data.drop(['Время замера MIN', 'key_x', 'Время замера', 'Время замера MAX', 'key_y', 'Время замера'], axis = 1)
work_data = work_data.rename(columns={"key_xxx": "key"})
work_data = work_data.dropna()

,key,Температура 1,Температура 2
0,1,1571.0,1613.0
1,2,1581.0,1602.0
2,3,1596.0,1599.0
3,4,1601.0,1625.0
4,5,1576.0,1602.0
...,...,...,...
2324,2495,1570.0,1591.0
2325,2496,1554.0,1591.0
2326,2497,1571.0,1589.0
2327,2498,1591.0,1594.0


In [18]:
data_arc['Конец нагрева дугой'] = pd.to_datetime(data_arc['Конец нагрева дугой'])
data_arc['Начало нагрева дугой']= pd.to_datetime(data_arc['Начало нагрева дугой'])
data_arc['Время нагрева'] = data_arc['Конец нагрева дугой'] - data_arc['Начало нагрева дугой']
data_arc['Отношение мощности'] = data_arc['Активная мощность'] / data_arc['Реактивная мощность']
data_arc['Время нагрева всего'] = (data_arc['Конец нагрева дугой'] - data_arc['Начало нагрева дугой']) / np.timedelta64(1, 's')

In [19]:
data_arc_power = data_arc.groupby('key').sum('Отношение мощности')
data_arc_power = data_arc_power.drop('Отношение мощности', axis = 1)
data_arc_power = data_arc_power.drop('Время нагрева всего', axis = 1)

,Активная мощность,Реактивная мощность
key,,
1,4.878147,3.183241
2,3.052598,1.998112
3,2.525882,1.599076
4,3.209250,2.060298
5,3.347173,2.252643
...,...,...
3237,3.932467,2.828459
3238,2.106529,1.681182
3239,4.666754,3.111617


In [20]:
data_arc_power['key'] = data_arc_power.index
work_data.reset_index(drop = True, inplace = True)
data_arc_power.reset_index(drop = True, inplace = True)
work_data = work_data.merge(data_arc_power, on='key', how='inner')  

In [21]:
count_n = pd.DataFrame([])
count_n= data_arc.groupby('key').count()
count_n['key'] = count_n.index
count_n.reset_index(drop = True, inplace = True)
count_n = count_n.drop(['Начало нагрева дугой', 'Конец нагрева дугой', 'Реактивная мощность', 'Время нагрева', 'Отношение мощности', 'Активная мощность'], axis = 1)
count_n = count_n.rename(columns={"Время нагрева всего": "Количество нагреваний"})
work_data = work_data.merge(count_n, on='key', how='inner')     

In [22]:
data_bulk = data_bulk.fillna(0)
data_wire = data_wire.fillna(0)

In [23]:
work_data = work_data.merge(data_bulk, on='key', how='inner')
work_data = work_data.merge(data_gas, on='key', how='inner')
work_data = work_data.merge(data_wire, on='key', how='inner')

,key,Температура 1,Температура 2,Активная мощность,Реактивная мощность,Количество нагреваний,Bulk 1,Bulk 2,Bulk 3,Bulk 4,...,Bulk 15,Газ 1,Wire 1,Wire 2,Wire 3,Wire 4,Wire 6,Wire 7,Wire 8,Wire 9
0,1,1571.0,1613.0,4.878147,3.183241,5,0.0,0.0,0.0,43.0,...,154.0,29.749986,60.059998,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
1,2,1581.0,1602.0,3.052598,1.998112,4,0.0,0.0,0.0,73.0,...,154.0,12.555561,96.052315,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
2,3,1596.0,1599.0,2.525882,1.599076,5,0.0,0.0,0.0,34.0,...,153.0,28.554793,91.160157,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
3,4,1601.0,1625.0,3.209250,2.060298,4,0.0,0.0,0.0,81.0,...,154.0,18.841219,89.063515,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
4,5,1576.0,1602.0,3.347173,2.252643,4,0.0,0.0,0.0,78.0,...,152.0,5.413692,89.238236,9.11456,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2324,2495,1570.0,1591.0,3.210690,2.360777,4,0.0,0.0,21.0,0.0,...,223.0,7.125735,89.150879,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
2325,2496,1554.0,1591.0,4.203064,2.810185,6,0.0,0.0,0.0,63.0,...,226.0,9.412616,114.179527,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
2326,2497,1571.0,1589.0,2.212379,1.851269,3,0.0,0.0,0.0,85.0,...,226.0,6.271699,94.086723,9.04800,0.0,0.0,0.0,0.0,0.0,0.0
2327,2498,1591.0,1594.0,3.408725,2.355428,5,0.0,0.0,90.0,0.0,...,207.0,14.953657,118.110717,0.00000,0.0,0.0,0.0,0.0,0.0,0.0


**План работы**

**Наша главная задача - предсказать финальную температуру. Исходя из данной цели мы выполним следующие задачи:**
- Проанализировать наши данные
- Обработать данные
- Сгенерировать дополнительные параметры
- Создать единый датафрейм для дальнейшей работы
- Разделить датафрейм на фичи/таргет и обучающую/валидационную выборки
- Выбрать лучшую модель на кросс-валидации/валидационной выборке и проверить качество этой модели на тестовой выборке

Проводим финальную чистку данных

In [25]:
work_data = work_data.fillna(0)
work_data = work_data[work_data['Температура 2'] !=0]
work_data = work_data.drop(['key'], axis = 1)

Делим данные на выборки

In [26]:
train_data, test_data = train_test_split(work_data, test_size=0.2, random_state=1)
data_train, data_valid = train_test_split(train_data, test_size=0.2, random_state=1)

In [27]:
data_train_target = data_train['Температура 2']
data_train_features = data_train.drop('Температура 2', axis=1)

data_valid_target = data_valid['Температура 2']
data_valid_features = data_valid.drop('Температура 2', axis=1)

test_data_target = test_data['Температура 2']
test_data_features = test_data.drop('Температура 2', axis=1)

Ниже тестируем различные модели в поисках лучшей.

In [28]:
model = LinearRegression()
model.fit(data_train_features, data_train_target)
result_LR = model.predict(data_valid_features)
mean_absolute_error(data_valid_target, result_LR)

6.098251461211146

In [29]:
model = Ridge(random_state=665)
model.fit(data_train_features, data_train_target)
result_R = model.predict(data_valid_features)
mean_absolute_error(data_valid_target, result_R)

6.098145499124539

In [30]:
model = RandomForestRegressor(max_depth=9, random_state=665)
model.fit(data_train_features, data_train_target)
result_RFR = model.predict(data_valid_features)
mean_absolute_error(data_valid_target, result_RFR)

5.694476791749159

In [31]:
model = LGBMRegressor(random_state=665)
model.fit(data_train_features, data_train_target)
result_RFR = model.predict(data_valid_features)
mean_absolute_error(data_valid_target, result_RFR)

5.675695447031286

In [32]:
model = CatBoostRegressor(random_state=665)
model.fit(data_train_features, data_train_target)
result_RFR = model.predict(data_valid_features)

Learning rate set to 0.043606
0:	learn: 11.2419024	total: 51.1ms	remaining: 51s
1:	learn: 11.1169287	total: 54ms	remaining: 26.9s
2:	learn: 10.9996149	total: 56.8ms	remaining: 18.9s
3:	learn: 10.8791861	total: 59.6ms	remaining: 14.9s
4:	learn: 10.7495530	total: 62.4ms	remaining: 12.4s
5:	learn: 10.6720471	total: 65.1ms	remaining: 10.8s
6:	learn: 10.5689475	total: 67.8ms	remaining: 9.62s
7:	learn: 10.4711008	total: 70.7ms	remaining: 8.77s
8:	learn: 10.3611599	total: 73.5ms	remaining: 8.09s
9:	learn: 10.2644908	total: 76.2ms	remaining: 7.54s
10:	learn: 10.1561446	total: 78.9ms	remaining: 7.1s
11:	learn: 10.0854530	total: 81.7ms	remaining: 6.73s
12:	learn: 9.9990899	total: 84.7ms	remaining: 6.43s
13:	learn: 9.9145773	total: 87.6ms	remaining: 6.17s
14:	learn: 9.8403236	total: 90.3ms	remaining: 5.93s
15:	learn: 9.7683988	total: 93.1ms	remaining: 5.72s
16:	learn: 9.7015838	total: 96.1ms	remaining: 5.56s
17:	learn: 9.6535961	total: 101ms	remaining: 5.5s
18:	learn: 9.5976874	total: 105ms	remai

In [33]:
mean_absolute_error(data_valid_target, result_RFR)

5.491454934638283

Видим, что лучшие результаты показывает CatBoost. Отправляем модель на финальным этап.

Финальная проверка качества модели.

In [34]:
result_RFR_test = model.predict(test_data_features)
mean_absolute_error(test_data_target, result_RFR_test)

5.8745989911086

Модель удовлетворяет нашим требованиям.

В результате мы выявили, что уменьшение количества данных положительно влияет на модель и не все классические метрики (в разрезе мощности) применимы и положительно влияют на модель.

Использованные признаки:
- Начальная температура.  Взяли начальную температуру

- Последняя температура.  Взяли последнюю измеренную температуру
- Активаня мощность.  Взяли активную мощность. 
- Реактивная мощность.  Взяли реактивную мощность
- Количество нагреваний.  Посчитали количество итераций нагревания.
- Bulk с разбивкой по итерациям.  Здесь мы взяли данные без группировок, так как группкировки типа суммирования показали себя хуже сырых данных.
- Газ.  Взяли объём газа для продувки.
- Wire с разбивкой по итерациям.  Здесь мы взяли данные без группировок, так как группкировки типа суммирования показали себя хуже сырых данных.

**Ниже параметры нашей лучшей модели.**

In [35]:
print(model.get_all_params())

{'nan_mode': 'Min', 'eval_metric': 'RMSE', 'iterations': 1000, 'sampling_frequency': 'PerTree', 'leaf_estimation_method': 'Newton', 'grow_policy': 'SymmetricTree', 'penalties_coefficient': 1, 'boosting_type': 'Plain', 'model_shrink_mode': 'Constant', 'feature_border_type': 'GreedyLogSum', 'bayesian_matrix_reg': 0.10000000149011612, 'force_unit_auto_pair_weights': False, 'l2_leaf_reg': 3, 'random_strength': 1, 'rsm': 1, 'boost_from_average': True, 'model_size_reg': 0.5, 'pool_metainfo_options': {'tags': {}}, 'subsample': 0.800000011920929, 'use_best_model': False, 'random_seed': 665, 'depth': 6, 'posterior_sampling': False, 'border_count': 254, 'classes_count': 0, 'auto_class_weights': 'None', 'sparse_features_conflict_fraction': 0, 'leaf_estimation_backtracking': 'AnyImprovement', 'best_model_min_trees': 1, 'model_shrink_rate': 0, 'min_data_in_leaf': 1, 'loss_function': 'RMSE', 'learning_rate': 0.043605998158454895, 'score_function': 'Cosine', 'task_type': 'CPU', 'leaf_estimation_itera